**Purpose of this project:**
Use regression techniques to predict sales price. 

**Procedures:**
1. Data preprocessing:
    a. Dealing with missing values
    b. Data transformation
    c. Handling categorical data
    
2. Feature selection: After comparing the results of selection based on VIF, random forest predictor importance, and correlations and pca, I decide to use correlation and pca to do feature selection. 1. keep continuous features that has significant correlation with the outcome. 2. Standardized data. 3.Use PCA to do dimension reduction.

3. Model Fitting: After trying the following methods: Linear Regression, Lasso regression, Ridge regression, Random Forest, XGBoost, SVR linear, I decided to use XGBoost.
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
house_price = pd.read_csv("../input/train.csv")
house_price.describe()

print(house_price.columns)
#(1460,81)
house_price.shape



 1.a
 Check missing data and the meaning of NA's .

In [ ]:
hp_NA = house_price.isnull().sum()
print(hp_NA[hp_NA!=0])

In [ ]:
def fillmissing(house_price):
    house_price['LotFrontage']=house_price.groupby('Street')['LotFrontage'].transform(
        lambda x: x.fillna(x.mean()))
    house_price['Alley']=house_price['Alley'].fillna('NoAlley')
    house_price['MasVnrType']=house_price['MasVnrType'].fillna('None')
    house_price['MasVnrArea']=house_price['MasVnrArea'].fillna(0)
    for col in ('BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2'):
        house_price[col]=house_price[col].fillna('NoBase')
    house_price['Electrical']=house_price['Electrical'].fillna(
        house_price['Electrical'].mode()[0])
    house_price['FireplaceQu']=house_price['FireplaceQu'].fillna('NoFireplace')
    for col in ('GarageType','GarageFinish','GarageQual','GarageCond'):
        house_price[col]=house_price[col].fillna('NoGarage')
    house_price['GarageYrBlt']=house_price['GarageYrBlt'].fillna(0)
    house_price['PoolQC']=house_price['PoolQC'].fillna('NoPool')
    house_price['Fence']=house_price['Fence'].fillna('NoFence')
    house_price['MiscFeature']=house_price['MiscFeature'].fillna('None')
    return house_price

house_price = fillmissing(house_price )
hp_NA = house_price.isnull().sum()
print(hp_NA[hp_NA!=0])
house_price.shape

In [ ]:
test = pd.read_csv("../input/test.csv")
test = fillmissing(test)
test['MSZoning']=test['MSZoning'].fillna(test['MSZoning'].mode()[0])
test['Utilities']=test['Utilities'].fillna(test['Utilities'].mode()[0])
test['Exterior1st']=test['Exterior1st'].fillna(test['Exterior1st'].mode()[0])
test['Exterior2nd']=test['Exterior2nd'].fillna(test['Exterior2nd'].mode()[0])
test['BsmtFinSF1']=test['BsmtFinSF1'].fillna(0)
test['BsmtFinSF2']=test['BsmtFinSF2'].fillna(0)
test['BsmtUnfSF']=test['BsmtUnfSF'].fillna(0)
test['TotalBsmtSF']=test['TotalBsmtSF'].fillna(0)
test['BsmtFullBath']=test['BsmtFullBath'].fillna(0)
test['BsmtHalfBath']=test['BsmtHalfBath'].fillna(0)
test['KitchenQual']=test['KitchenQual'].fillna(test['KitchenQual'].mode()[0])
test['GarageCars']=test['GarageCars'].fillna(0)
test['GarageArea']=test['GarageArea'].fillna(0)
test['SaleType']=test['SaleType'].fillna(test['SaleType'].mode()[0])
test['Functional']=test['Functional'].fillna(test['Functional'].mode()[0])
test_NA = test.isnull().sum()
print(test_NA[test_NA!=0])
test_id=test.Id

1.b
Data transformation

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

sns.set(color_codes=True)
sns.distplot(house_price.SalePrice,kde=True)
plt.title("SalePrice distribution")
plt.show()
# The distribution of y violates the assumption of normal distribution
# Do log transformation on y
house_price['log_y'] =np.log1p(house_price.SalePrice) 
sns.distplot(house_price.log_y,kde=True)
plt.title("Log(1+SalePrice) distribution")
plt.show()

Find correlations and its p value between continuous features and the outcome variable.

In [ ]:
from scipy.stats import pearsonr

x_r = house_price[["LotFrontage","LotArea", "OverallQual", 
                   "OverallCond", "YearBuilt", "YearRemodAdd", 
                   "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", 
                   "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", 
                   "2ndFlrSF", "LowQualFinSF", "GrLivArea", 
                   "BsmtFullBath", "BsmtHalfBath", "FullBath", 
                   "HalfBath", "TotRmsAbvGrd", "Fireplaces", 
                   "GarageYrBlt", "GarageCars", "GarageArea", 
                   "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", 
                   "3SsnPorch", "ScreenPorch", "PoolArea", 
                   "MiscVal", "MoSold", "YrSold"]]
y_r = house_price.log_y

a = pd.DataFrame(columns=["r","p"], index = x_r.columns)
for col in x_r.columns:
    b = pearsonr(x_r[col],y_r) 
    if b[1] < 0.05:
        a.loc[col] = pd.Series({"r":b[0], "p":b[1]})
        print (col, " r:", b[0], " p:", b[1])
a.sort_values(by = ["r"],ascending=False)


Delete continuous variables that have no significant (p>0.01) correlation with the outcome variable. 
Transform categorical variable to dummy variables.

In [ ]:
hp_drop = house_price.drop(["Id","OverallCond","BsmtFinSF2","LowQualFinSF",
                  "BsmtHalfBath","MiscVal","YrSold","SalePrice"],axis=1)

tt_drop = test.drop(["Id","OverallCond","BsmtFinSF2","LowQualFinSF",
                  "BsmtHalfBath","MiscVal","YrSold"],axis=1)

X,y = hp_drop.drop(['log_y'],axis=1),hp_drop.log_y

alldata = pd.concat([X,tt_drop],ignore_index=True,sort=False)
all_one_hot = pd.get_dummies(alldata)
train = all_one_hot.loc[0:(X.shape[0]-1),]
tt = all_one_hot.loc[X.shape[0]:(all_one_hot.shape[0]-1),]

print(train.shape,tt.shape)

Standardize remaining features and use PCA to transform data. Only keep 6 principal components.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# 1st component:9.83744091e-01, 2nd: 5.08583965e-03, 3rd: 3.38481930e-03, 
# 4th:2.77136722e-03, 5th: 1.96914612e-03, 6th: 1.75122582e-03. 

pipe_pc = Pipeline([("scl", StandardScaler()),
                    ("pca", PCA(n_components=6))])

train_pc = pipe_pc.fit_transform(train)
test_pc = pipe_pc.transform(tt)



Create a function to do 5-fold cross validation.

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

def rmse_cv(model, X, y):
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=100)
    rmse = np.sqrt(-cross_val_score(model,X,y, cv=cv, scoring="neg_mean_squared_error"))
    return(rmse)

Hyperparameter tuning.

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

rgr = XGBRegressor(random_state=100)
para_range = np.linspace(start= 0.0001,stop=1, num=100)
params =[{'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
         'learning_rate': para_range}]
rgr_random = GridSearchCV(estimator = rgr, 
                        param_grid = params, cv = 5, 
                        verbose=2,  
                        scoring = 'neg_mean_squared_error',n_jobs = -1)

rgr_random.fit(train_pc,y)
print('best parameters: %s' %rgr_random.best_params_)
print('rmse: %.3f' %np.sqrt(-rgr_random.best_score_))

In [ ]:
Compare the results between the tuned model and base model. Use tuned model to make prediction.

In [ ]:

from xgboost import XGBRegressor
xg_best = XGBRegressor(n_estimator=200, learning_rate=0.0809,random_state=100)
score = rmse_cv(xg_best,train_pc,y)

print('train: mean rmse: %.3f, std rmse: %.3f' %(score.mean(),score.std()))


xg_base=XGBRegressor(n_estimator=200,random_state=100)
score = rmse_cv(xg_base,train_pc,y)

print(score.mean(), score.std())


Xgboost has the lowest rmse, so we use xgboost to predict the test data. After than, transform the log y back to y and export data.

In [ ]:
xg_best = XGBRegressor(n_estimator=200, learning_rate=0.0809,random_state=100)
xg_best.fit(train_pc,y)
y_pred = xg_best.predict(test_pc)
y_test = np.expm1(y_pred)


In [ ]:

sub = pd.DataFrame()
sub['Id'] = test_id
sub['SalePrice'] = y_test
sub.to_csv('submission3.csv',index=False)
